In [1]:
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Flatten, TimeDistributed, Dropout, LSTMCell, RNN, Bidirectional, Concatenate, Layer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.utils import tf_utils
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import pickle
import tensorflow_hub as hub

from sklearn.model_selection import train_test_split

import unicodedata
import re
import os
import time
import shutil
import requests
import tarfile
import glob

import argparse
from tokenize import tokenize, untokenize, COMMENT, STRING, NEWLINE, ENCODING, ENDMARKER, NL, INDENT, NUMBER
from io import BytesIO
import json

import pandas as pd
import numpy as np
import string, os
tf.__version__

'2.4.0'

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

In [13]:
with open("train_sent.txt", "r") as fp: 
    train_sent = fp.read().splitlines()
with open("test_sent.txt", "r") as fp:
    test_sent = fp.read().splitlines()
with open("full_corpus.txt", "r") as fp:
    full_corpus = fp.read().splitlines()

In [5]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer
tokenizer = Tokenizer(models.BPE())

In [6]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

In [8]:
tokenizer.pre_tokenizer.pre_tokenize_str(train_sent[1])

[('n', (0, 1)),
 ('Ġ,', (1, 3)),
 ('Ġ*', (3, 5)),
 ('Ġa', (5, 7)),
 ('Ġ=', (7, 9)),
 ('Ġmap', (9, 13)),
 ('Ġ(', (13, 15)),
 ('Ġint', (15, 19)),
 ('Ġ,', (19, 21)),
 ('Ġopen', (21, 26)),
 ('Ġ(', (26, 28)),
 ('Ġ<', (28, 30)),
 ('NUM', (30, 33)),
 ('_', (33, 34)),
 ('LIT', (34, 37)),
 (':', (37, 38)),
 ('0', (38, 39)),
 ('>', (39, 40)),
 ('Ġ)', (40, 42)),
 ('Ġ.', (42, 44)),
 ('Ġread', (44, 49)),
 ('Ġ(', (49, 51)),
 ('Ġ)', (51, 53)),
 ('Ġ.', (53, 55)),
 ('Ġsplit', (55, 61)),
 ('Ġ(', (61, 63)),
 ('Ġ)', (63, 65)),
 ('Ġ)', (65, 67)),
 ('Ċ', (67, 68))]

In [21]:
batch_size=1000
all_texts = [full_corpus[i : i+batch_size] for i in range(0, len(full_corpus), batch_size)]

In [22]:
def batch_iterator():
    for i in range(0, len(full_corpus), batch_size):
        yield full_corpus[i : i + batch_size]

In [43]:
literals = ['name', '\\n', 'U+0020', 'True', 'a', 'type', '__main__', 'id', '/', '.', 'w', 'utf-8', 'b', 'foo', '1', 'r', 'default', 'U+002C', '-', 'data', '0', '%s', 'x', 'blank', ':', 'value', 'max_length', 'c', '_', 'test', 'A', 'store_true', 'description', '*', 'null', 'title', 'y', 'U+002CU+0020', 'true', 'key', 'bar', '=', 'string', 'status', '2', 'rb', 'to', 'B', 'url', 'd', '"', 'version', 'str', 'abc', 'int', 'GET', 'text', 'error', 'user', 'C', ')', 'wb', '\\t', '+', '\\\\', '#', '>', 'f', ';', 'path', 'ascii', 'POST', '?', '3', 'False', 'D', '|', 'label', 's', '(', 'message', "'", 'i', 'primary_key', '<', 'Meta', 'N', 'e', 'object_name', 'password', 'index', '..', 'code', 'class', 'username', 'F', 'size', '}', 'n', 'X', 'Y', 'file', 'z', '[', 'store', 'none', 'html', ']', 'utf8', 'p', 'strict', '\\r\\n', 'end', 'all', 'Name', 'state', 'produces', 'm', 't', 'I', 'result', 'bool', 'date', '{}', '\\x00', 'hello', 'start', 'left', 'email', 'ignore', 'S', '@', 'L', 'false', 'action', '{', 'related_name', 'time', 'count', 'v', '4', 'relu', 'win32', 'port', 'h', '#pop', 'source', 'Content-Type', 'application/json', '&', 'src', 'O', '-c', 'H', 'o', 'g', 'None', 'E', 'M', 'host', 'body', '%', 'info', 'image', 'float', 'root', 'l', 'T', 'replace', 'k', '\\ufffe', '100', '5', '$', 'right', 'content', 'q', 'args', 'u', 'address', '\\r', 'list', 'target', 'http', 'P', 'object', 'yes', '1.0', 'R', 'width', 'localhost', '127.0.0.1', 'train', '[?]', 'OK', 'location', 'filename', '!', 'success', 'input']
tokens = ["<|endoftext|>","<EOL>","<s>","</s>","<NUM_LIT>","<STR_LIT>"]
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=literals+tokens)
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

In [44]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

In [45]:
from transformers import GPT2TokenizerFast

new_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

In [46]:
new_tokenizer.save_pretrained("code-tokenizer-scratch")

('code-tokenizer-scratch/tokenizer_config.json',
 'code-tokenizer-scratch/special_tokens_map.json',
 'code-tokenizer-scratch/vocab.json',
 'code-tokenizer-scratch/merges.txt',
 'code-tokenizer-scratch/added_tokens.json',
 'code-tokenizer-scratch/tokenizer.json')

In [ ]:
!pip install 